<a href="https://colab.research.google.com/github/patmadp16/praprocessing-text.python/blob/main/Preprocessing_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect google/drive

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

# Read Dataset 

In [ ]:
import pandas as pd
import numpy as np

#pd.set_option("max_columns",None)
#komentar = pd.read_csv("/content/drive/MyDrive/NLP/layanan.csv",usecols=['Text'],encoding="ISO-8859-1")
#menampilkan data komentar kkb
komentar=pd.read_csv('/content/drive/MyDrive/NLP/layanan.csv',error_bad_lines=False, engine="python")
pd.set_option('display.max_colwidth', -1)
#using only v1 and v2 column
#data.head()
komentar.head()

In [ ]:
#mengecek data kosong
komentar.isna().sum()

In [ ]:
#menghapus baris yang datanya kosong
komentar.dropna(inplace=True)
komentar.isnull().sum()

#Case Folding

In [ ]:
#------Case Folding------
komentar['Comments_CaseFolding']=komentar['Text'].str.lower()
komentar[['Comments_CaseFolding','Text']].head()

In [ ]:
import re
def remove_pattern(text,pattern):
  r = re.findall(pattern,text)
  for i in r:
    text = re.sub(i, '' , text)
  return text

komentar['Comments_CaseFolding']=np.vectorize(remove_pattern)(komentar['Comments_CaseFolding'],"#[\w]*")
komentar['Comments_CaseFolding']=np.vectorize(remove_pattern)(komentar['Comments_CaseFolding'],"@[\w]*")
komentar[['Comments_CaseFolding','Text']].head(10)

In [ ]:
#remove url
import re
def clean_links(text):
  text = re.sub(r"http\S+", "", text)
  return text

komentar['Comments_Text']=komentar['Comments_CaseFolding'].apply(clean_links)
komentar[['Comments_Text','Comments_CaseFolding']].head(5)

# Remove Char, Punctuation, DoubleWord



In [ ]:
import re
def remove_emoticon(text):
  text = text.encode('ascii',"replace").decode('ascii')
  return text

komentar['Comments_dText'] = komentar['Comments_Text']  

Remove Char, Number, Double World

In [ ]:

#text cleaning
import re

def clean_text(text):
  #Remove RT/ b'
  text = text.replace("b'","")

  #Remove \n, \t, \r
  text = re.sub(r"\\n","",text)
  text = re.sub(r"\\r"," ",text)
  text = re.sub(r"\\t"," ",text)

  text = re.sub("\s+"," ",text)

  #Mengganti &amp dengan &
  text = re.sub(r"&amp;"," ",text)
  
  text=text.lower()

  #Remove tanda baca
  text = re.sub(r"[?!.,;:#@_+()|\/<>]","",text)
  text = text.replace("https", '').replace("http",'')

  return text
  
def remove_emoticon(text):
  text = text.encode('ascii',"replace").decode('ascii')
  return text  


#tweets = remove_emoticon(dataset.text.str)
komentar['Comments_Text'] = komentar['Comments_Text'].apply(clean_text)
#komentar['Comments_Text']= komentar['Comments_Text'].apply(remove_emoticon2)
komentar['Comments_Text']= komentar['Comments_Text'].str.replace("[^a-zA-Z#][\a]"," ")
#komentar['Comments_Text']= komentar['Comments_Text'].str.replace(u'\xe2','a')

komentar[['Comments_Text','Comments_CaseFolding']].head(5) 

In [ ]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

#library that contains punctuation
string.punctuation

def remove_char(text):
  return text.replace(".", " ")

#Menghapus tanda baca
def remove_punctuation(text):
    punctuation_comments="".join([i for i in text if i not in string.punctuation])
    text.replace(punctuation_comments," ")
    return punctuation_comments

#Meghapus emoji
def deEmojify(text):
    return text.encode('ascii', 'ignore').decode('ascii')

#Menghapus angka
def remove_number(text):
    return  re.sub(r"\d+", "", text)

komentar['RemoveChar'] = komentar['Comments_Text'].apply(remove_char).apply(lambda x:remove_punctuation(x)).apply(remove_number).apply(deEmojify)
komentar[['RemoveChar','Comments_Text']].head()

In [ ]:
# Menghapus double word
from collections import OrderedDict
komentar['RemoveDoubleWord'] = (komentar['RemoveChar'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))

komentar[['RemoveChar','RemoveDoubleWord']].head(5)

# Tokenizing

In [ ]:
#defining function for tokenization
import re
import nltk
nltk.download('punkt')

def tokenization(text):
    tokens = re.split('W+',text)
    return tokens

def word_tokenize_wrapper(text):
    return word_tokenize(text)

#applying function to the column
komentar['tokenizing']= komentar['RemoveDoubleWord'].apply(word_tokenize_wrapper)
komentar[['tokenizing','RemoveDoubleWord']].head(5)

# Normalisasi slang word

In [ ]:
normalizad_word = pd.read_excel("/content/drive/MyDrive/NLP/Normalisasi_word.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

komentar['Normal_token'] = komentar['tokenizing'].apply(normalized_term)

komentar[['tokenizing','Normal_token']].head(10)

# Stopword

In [ ]:
#import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# read txt stopword using pandas
txt_stopword = pd.read_csv("/content/drive/MyDrive/NLP/stopword_combine.txt", names= ["stopwords"],header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
  output = [i for i in words if i not in list_stopwords]
  return output

def remove_stopwords(text):
    output= [i for i in text if i not in stopwords_english]
    return output

komentar['Comments_stopwordfix'] =komentar['Normal_token'].apply(stopwords_removal)
komentar[["Normal_token","Comments_stopwordfix"]].head(5)

In [ ]:
!pip install Sastrawi
# import StemmerFactory class
#importing the Stemming function from nltk library
#from nltk.stem.porter import PorterStemmer
#defining the object for stemming
#porter_stemmer = PorterStemmer()
#defining a function for stemming
#def stemming(text):
#  output = [porter_stemmer.stem(word) for word in text]
#  return output
#data['msg_stemmed']=data['no_sw_msg'].apply(lambda x: stemming(x))
#komentar['Stemming'] =komentar['Comments_stopwordfix'].apply(lambda x: stemming(x))

#komentar.head()

In [ ]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
#import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}
for document in komentar['Comments_stopwordfix']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]


komentar['Stemming2'] =komentar['Comments_stopwordfix'].apply(get_stemmed_term)
komentar[['Comments_stopwordfix','Stemming2']].head(10)


7115


In [ ]:
komentar.to_csv("comment_kkb_process.csv")